In [2]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [3]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [4]:
df = pd.read_csv('Condo_buyout_map_template_popup_pin - Sheet1.csv',dtype='str')

In [5]:
df = df.fillna('Not Available')

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [8]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [9]:
# print(df['image_links'].iloc[0])

In [10]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[1]

'https://static.therealdeal.com/wp-content/uploads/2023/07/Mast-Capital-Sued-Over-Miami-Beach-Condo-Buyout_Main-1400x875.jpg'

In [11]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [12]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2023/05...
1    <a href="https://therealdeal.com/miami/2023/07...
2    <a href="https://therealdeal.com/miami/2023/03...
3    <a href="https://therealdeal.com/miami/2023/06...
4    <a href="https://therealdeal.com/miami/2023/05...
5    <a href="https://therealdeal.com/miami/2023/06...
6    <a href="https://therealdeal.com/miami/2022/10...
Name: description_link, dtype: object

In [13]:
df

,full_address,name,buyers,description,price,status,currently facing legal challenge?,story_link,geocoded,lat,lon,image_links,short_description,remaining_desc,description_link
0,2121 North Bayshore Drive in Miami,Biscayne 21,Two Roads Development,Two Roads paid $150 million for the majority o...,$150M,Completed,Yes,https://therealdeal.com/miami/2023/05/24/edgew...,"(25.7975549, -80.18641679999999)",25.797555,-80.186417,https://static.therealdeal.com/wp-content/uplo...,Two Roads paid $150,"million for the majority of Biscayne 21, a 13...","<a href=""https://therealdeal.com/miami/2023/05..."
1,5313 Collins Avenue,Amethyst,Mast Capital; Terra,Mast Capital has been pursuing a buyout of Ame...,Not Available,In Contract; Offers made,Yes,https://therealdeal.com/miami/2023/07/14/mast-...,"(25.831677, -80.120971)",25.831677,-80.120971,https://static.therealdeal.com/wp-content/uplo...,Mast Capital has,"been pursuing a buyout of Amethyst, an 11-sto...","<a href=""https://therealdeal.com/miami/2023/07..."
2,5445 Collins Avenue in Miami Beach,Castle Beach Club,Terra,David Martin’s Terra offered owners at the Cas...,$500M,Offers made,No,https://therealdeal.com/miami/2023/03/20/terra...,"(25.8343741, -80.1202116)",25.834374,-80.120212,https://static.therealdeal.com/wp-content/uplo...,David Martin’s Terra,offered owners at the Castle Beach Club condo...,"<a href=""https://therealdeal.com/miami/2023/03..."
3,6345 Collins Avenue in Miami Beach,Casablaca,Richard Meruelo,"In June, Richard Meruelo backed out of a $200 ...",$200M,Canceled,No,https://therealdeal.com/miami/2023/06/29/merue...,"(25.8452431, -80.1198764)",25.845243,-80.119876,https://static.therealdeal.com/wp-content/uplo...,"In June, Richard",Meruelo backed out of a $200 million buyout o...,"<a href=""https://therealdeal.com/miami/2023/06..."
4,"8000, 8010 and 8020 East Drive in North Bay Vi...",Biscayne Sea Club,Macklowe Properties,New York developer Harry Macklowe has been ass...,$48M,In litigation,Yes,https://therealdeal.com/miami/2023/05/19/mackl...,"(25.8545286, -80.1575787)",25.854529,-80.157579,https://static.therealdeal.com/wp-content/uplo...,New York developer,Harry Macklowe has been assembling a large wa...,"<a href=""https://therealdeal.com/miami/2023/05..."
5,3611 Collins Avenue in Miami Beach,Westgate South Beach Oceanfront Resort,"13th Floor Investments, Opera Acquisitions",13th Floor Investments joined with Opera Acqui...,$73M,Closed,No,https://therealdeal.com/miami/2023/06/27/13th-...,"(25.8098006, -80.1230121)",25.809801,-80.123012,https://static.therealdeal.com/wp-content/uplo...,13th Floor,Investments joined with Opera Acquisitions to...,"<a href=""https://therealdeal.com/miami/2023/06..."
6,2730 South Ocean Boulevard and 2720 South Ocea...,The Ambassador Palm Beach Hotel & Residences a...,OKO Group and Cain International,"In October, Vlad Doronin’s OKO Group and partn...",$147M,Closed,No,https://therealdeal.com/miami/2022/10/21/doron...,"(26.6227937, -80.0373591)",26.622794,-80.037359,https://static.therealdeal.com/wp-content/uplo...,"In October, Vlad",Doronin’s OKO Group and partner Cain Internat...,"<a href=""https://therealdeal.com/miami/2022/10..."


In [14]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    name = df['name'].iloc[i]
    buyers = df['buyers'].iloc[i]
    price = df['price'].iloc[i]
    status = df['status'].iloc[i]
    legal_c = df['currently facing legal challenge?'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Name: </strong>{}'''.format(name) + '''<br>
    <strong>Buyers: </strong>{}'''.format(buyers) + '''<br>
    <strong>Price: </strong>{}'''.format(price) + '''<br>
    <strong>Status: </strong>{}'''.format(status) + '''<br>
    <strong>Lawsuits? </strong>{}'''.format(legal_c) + '''<br>
    {}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [15]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Condo Buyouts')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [16]:
m.save('index.html')

## Map URL Snagger

In [17]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_condo_buyouts
